<a href="https://colab.research.google.com/github/izzul112/Submission-Image_Classification/blob/master/Submission_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# melakukan ekstraksi pada file zip
import zipfile,os
local_zip = '/content/gdrive/My Drive/dataset/train.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()



In [4]:
base_dir = '/tmp'
train_dir = os.path.join(base_dir, 'train')

In [5]:
os.listdir('/tmp/train')

['NORMAL', 'DME', 'CNV', 'DRUSEN']

In [6]:
# membuat direktori CNV pada direktori data training
train_CNV_dir = os.path.join(train_dir, 'CNV')

# membuat direktori DME pada direktori data training
train_DME_dir = os.path.join(train_dir, 'DME')

# membuat direktori DRUSEN pada direktori data training
train_DRUSEN_dir = os.path.join(train_dir, 'DRUSEN')

# membuat direktori NORMAL pada direktori data training
train_NORMAL_dir = os.path.join(train_dir, 'NORMAL')

# membuat direktori CNV pada direktori data validasi
validation_CNV_dir = os.path.join(train_dir, 'CNV')

# membuat direktori DME pada direktori data validasi
validation_DME_dir = os.path.join(train_dir, 'DME')

# membuat direktori DRUSEN pada direktori data validasi
validation_DRUSEN_dir = os.path.join(train_dir, 'DRUSEN')

# membuat direktori NORMAL pada direktori data validasi
validation_NORMAL_dir = os.path.join(train_dir, 'NORMAL')

In [7]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest',
                    validation_split=0.2)

In [8]:
train_generator = train_datagen.flow_from_directory(
        train_dir,  # direktori data latih
        target_size=(200, 300),  # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4,
        # karena kita merupakan masalah klasifikasi 2 kelas lebih maka menggunakan class_mode = 'categorical'
        class_mode="categorical",
        subset='training'
        )

validation_generator = train_datagen.flow_from_directory(
        train_dir, # direktori data validasi
        target_size=(200, 300), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4,
        # karena kita merupakan masalah klasifikasi 2 kelas lebih maka menggunakan class_mode = 'categorical'
        class_mode='categorical',
        subset='validation'
        )

Found 66788 images belonging to 4 classes.
Found 16696 images belonging to 4 classes.


In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(200, 300, 4)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.RMSprop(),
              metrics=['accuracy'])

In [11]:
model.fit(
      train_generator,
      steps_per_epoch=25,  # berapa batch yang akan dieksekusi pada setiap epoch
      epochs=20,
      validation_data=validation_generator, # menampilkan akurasi pengujian data validasi
      validation_steps=5,  # berapa batch yang akan dieksekusi pada setiap epoch
      verbose=2)

Epoch 1/20


InvalidArgumentError: ignored